In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

# Transfer Learning with PyTorch

In [13]:
import time
import datetime
import torch
import torchvision
from torch.utils.data import DataLoader
from torchvision import transforms
import torch.nn as nn
import torch.nn.functional as F
from PIL import Image
from IPython.display import display

## Declaring Dataloader [load data in NN a/c to batch size (if given)]

In [2]:
ris = resize_image_size = 64
train_data_path="/home/garvit/work_area/zone/Anaconda_WorkSpace/jupyter/PyTorch practice/PyTorch Book/chapter2/dataset/training_set/"
test_data_path="/home/garvit/work_area/zone/Anaconda_WorkSpace/jupyter/PyTorch practice/PyTorch Book/chapter2/dataset/test_set/"

# declaring some trasformations on images for NN feed i.e Scaling images, converting into tensors e.t.c
transformations = transforms.Compose([transforms.Resize((ris,ris)),
                                 transforms.ToTensor(),
                                 transforms.Normalize(mean= [0.485, 0.456, 0.406],
                                                      std = [0.229, 0.224, 0.225])])

# applying above transformations
train_data = torchvision.datasets.ImageFolder(root=train_data_path, transform=transformations)
# val_data   = torchvision.datasets.ImageFolder(root = val_data_path, transform=transforms) 
test_data  = torchvision.datasets.ImageFolder(root = test_data_path, transform=transformations)


# declaring DataLoader (load data in NN a/c to batch size if given)
BatchSize = 48
train_data_loader = DataLoader(dataset=train_data, batch_size = BatchSize, shuffle=True )
# val_data_loader = DataLoader(dataset=val_data, batch_size = BatchSize)
test_data_loader = DataLoader(dataset=test_data, batch_size = BatchSize, shuffle=True)


## Loading ResNet

In [36]:
torch.hub.list('pytorch/vision')

Using cache found in /home/garvit/.cache/torch/hub/pytorch_vision_master


['alexnet',
 'deeplabv3_resnet101',
 'densenet121',
 'densenet161',
 'densenet169',
 'densenet201',
 'fcn_resnet101',
 'googlenet',
 'inception_v3',
 'mobilenet_v2',
 'resnet101',
 'resnet152',
 'resnet18',
 'resnet34',
 'resnet50',
 'resnext101_32x8d',
 'resnext50_32x4d',
 'shufflenet_v2_x0_5',
 'shufflenet_v2_x1_0',
 'squeezenet1_0',
 'squeezenet1_1',
 'vgg11',
 'vgg11_bn',
 'vgg13',
 'vgg13_bn',
 'vgg16',
 'vgg16_bn',
 'vgg19',
 'vgg19_bn',
 'wide_resnet101_2',
 'wide_resnet50_2']

In [38]:
model = torch.hub.load('pytorch/vision', 'resnet50', pretrained=True )

Using cache found in /home/garvit/.cache/torch/hub/pytorch_vision_master
Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /home/garvit/.cache/torch/checkpoints/resnet50-19c8e357.pth
100.0%


In [4]:
transfer_model = torch.hub.load('pytorch/vision', 'resnet50', pretrained=True )

Using cache found in /home/garvit/.cache/torch/hub/pytorch_vision_master


## freezing the layers (except BatchNorm)

In [5]:
for name, params in transfer_model.named_parameters():
    if 'bn' not in name:
        params.requires_grad = False

## Replacing FC layers of ResNet with the custom one

In [6]:
transfer_model.fc = nn.Sequential(
                            nn.Linear(transfer_model.fc.in_features, out_features=500),
                            nn.ReLU(),
                            nn.Dropout(),

                            nn.Linear(500, 2))

## Create an optimizer

In [8]:
from torch.optim import Adam, SGD
optimizer = SGD(params=transfer_model.parameters(), lr=0.01)

## Checking for GPU

In [10]:
if torch.cuda.is_available():
    device = torch.device("cuda") 
else:
    device = torch.device("cpu")

transfer_model.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

## Defining Training Function 

In [11]:
def train(model, optimiser, loss_fn, train_loader, val_loader, epochs=1, device='cpu'):
    for epoch in range(epochs):
            training_loss = 0
            valid_loss = 0
            model.train()
            for batch in train_loader:
#                 print(f"training batch: {batch}")
                optimizer.zero_grad()
                inputs, target = batch
#                 print(inputs)
                inputs = inputs.to(device)
                target = target.to(device)
                output = model(inputs)
                loss = loss_fn(output, target)
                loss.backward()
                optimizer.step()
                training_loss += loss.data.item()
            training_loss /= len(train_loader.dataset)
            
            model.eval()
            num_correct = 0
            num_examples = 0
            for batch in val_loader:
                inputs, target = batch
                inputs = inputs.to(device)
                output = model(inputs)
                loss = loss_fn(output, target)
                valid_loss += loss.data.item()
                correct = torch.eq(torch.max(F.softmax(output), dim=1)[1], target).view(-1)
                num_correct += torch.sum(correct).item()
                num_examples += correct.shape[0]
            valid_loss /= len(val_loader.dataset)

            print(f'Epoch: {epoch+1},  Training Loss: {round(training_loss,3)}, \
            Validation Loss: {round(valid_loss,3)}, \
            accuracy = {round(num_correct / num_examples,2)}')            
            
    

In [14]:
start_time = time.time() 
train(transfer_model, optimizer,nn.CrossEntropyLoss(), train_data_loader,test_data_loader, epochs=5, device=device)
time_took = f"{time.time() - start_time!r}"
time_took = str(datetime.timedelta(seconds=round(float(time_took),2))).rstrip('0')
print(f"Training time: {time_took}")



/home/garvit/anaconda3/envs/UVA/lib/python3.7/site-packages/ipykernel_launcher.py:29: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Epoch: 1,  Training Loss: 0.009,             Validation Loss: 0.008,             accuracy = 0.82
Epoch: 2,  Training Loss: 0.008,             Validation Loss: 0.007,             accuracy = 0.85
Epoch: 3,  Training Loss: 0.007,             Validation Loss: 0.007,             accuracy = 0.85
Epoch: 4,  Training Loss: 0.006,             Validation Loss: 0.007,             accuracy = 0.86
Epoch: 5,  Training Loss: 0.005,             Validation Loss: 0.007,             accuracy = 0.87
Training time: 0:25:38.4
